In [1]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd


wttj_url = 'https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2'
page = requests.get(wttj_url)
page.text 

soup = bs(page.text, "html.parser")
#if page.status_code == 200:
    #print(f"reussi de la requête. Code de statut: {page.status_code}")
#else:
   #@ print(f"Échec de la requête. Code de statut: {page.status_code}")
#soup

In [2]:
def get_info(link):

    page = requests.get(link)
    soup = bs(page.text, "html.parser")
    company = soup.find('span', attrs = {'class' : "sc-empnci hmOCpj wui-text"}).string
    job = soup.find('h2', attrs = {'class': "sc-empnci cYPTxs wui-text"}).string
    lieu = soup.find('span', attrs = {'class': "sc-1eoldvz-2 kGQqJy"}).string
    spans = soup.find_all('span', attrs = {'class': "sc-bXCLTC kJcLKT"})
    try:
        if spans:
            start = spans[1].find_next("span").string
    except:
        start = "Date de prise de poste non trouvée"
    try : 
        if soup.find("time").has_attr('datetime'):
            debut= soup.find("time").string
    except : 
            debut = "Non trouvé"
        
    col = soup.find_all('span', attrs = {'class': "sc-empnci klToKr wui-text"})
    domaine = col[0].find_next("span").string
    line = [company,job,lieu,domaine,debut,link]
    return line
get_info('https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2')

['Hunteed',
 'Customer Success Manager B2C H/F',
 'Paris',
 'SaaS / Cloud Services, Recrutement, Digital',
 'il y a 8 heures',
 'https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2']

In [3]:
jobs_list = []
for i in range(1,35):
    all_jobs = "https://www.welcometothejungle.com/fr/jobs?query=communication&refinementList%5Boffices.country_code%5D%5B%5D=FR&page="+str(i)
    page = requests.get(all_jobs)
    soup = bs(page.text,'html.parser')
    divs = soup.find_all('div',class_="sc-bXCLTC sc-emIrwa bsJILN sc-6i2fyx-4 bpkkNp")

In [4]:
for div in divs : 
        a = div.find("a")
        if ("/fr/companies" in a["href"]) and (a["href"] not in jobs_list):
            jobs_list.append(a["href"])
            
            
jobs_list = list(set(jobs_list)) # évite les doublons
#len(jobs_list)

print(len(jobs_list))


0


In [5]:
base = "https://www.welcometothejungle.com"
df = pd.DataFrame(columns=["Company","Job","Lieu","Domaine","Mise en ligne","Link"])

for job in jobs_list : 
    #print("Ofrre ",i)
    row = get_info(base+job)
    a = pd.DataFrame([row],columns=["Company","Job","Lieu","Domaine","Mise en ligne","Link"])
    df = pd.concat([df,a],ignore_index=True)
    
    #print("---------------------------")
df = df.drop_duplicates().reset_index(drop=True)
df

,Company,Job,Lieu,Domaine,Mise en ligne,Link


In [6]:
df.to_csv(r'./welcome_to_the¨jungle.csv', index=False, header=True)